In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import pytesseract
image = Image.open('/home/vdubyna/PycharmProjects/pdf-data-extraction/photo_2023-02-23_20-36-25.jpg')
width, height = image.size
w_scale = 1000 / width
h_scale = 1000 / height

ocr_df = pytesseract.image_to_data(image, lang='ukr', output_type='data.frame')
ocr_df = ocr_df.dropna() \
    .assign(left_scaled=ocr_df.left * w_scale,
            width_scaled=ocr_df.width * w_scale,
            top_scaled=ocr_df.top * h_scale,
            height_scaled=ocr_df.height * h_scale,
            right_scaled=lambda x: x.left_scaled + x.width_scaled,
            bottom_scaled=lambda x: x.top_scaled + x.height_scaled)

float_cols = ocr_df.select_dtypes('float').columns
ocr_df[float_cols] = ocr_df[float_cols].round(0).astype(int)
ocr_df = ocr_df.replace(r'^\s*$', np.nan, regex=True)
ocr_df = ocr_df.dropna().reset_index(drop=True)
ocr_df[:20]


In [ ]:
draw = ImageDraw.Draw(image)

for idx, row in ocr_df.iterrows():
    x, y, w, h = row['left'], row['top'], row['width'], row['height']  # the row comes in (left, top, width, height) format
    draw.rectangle([x, y, x + w, y + h], width=2)

image

In [ ]:
import cv2
import pytesseract
import numpy as np

img = cv2.imread('/home/vdubyna/PycharmProjects/pdf-data-extraction/photo_2023-02-23_20-36-25.jpg')

width, height = image.size
w_scale = 1000 / width
h_scale = 1000 / height

def convert_grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def blur(img, param):
    return cv2.medianBlur(img, param)

def treshold(img):
    return cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

img = convert_grayscale(img)
img = treshold(img)

ocr_df = pytesseract.image_to_data(image, lang='ukr', output_type='data.frame')
ocr_df = ocr_df.dropna() \
    .assign(left_scaled=ocr_df.left * w_scale,
            width_scaled=ocr_df.width * w_scale,
            top_scaled=ocr_df.top * h_scale,
            height_scaled=ocr_df.height * h_scale,
            right_scaled=lambda x: x.left_scaled + x.width_scaled,
            bottom_scaled=lambda x: x.top_scaled + x.height_scaled)

float_cols = ocr_df.select_dtypes('float').columns
ocr_df[float_cols] = ocr_df[float_cols].round(0).astype(int)
ocr_df = ocr_df.replace(r'^\s*$', np.nan, regex=True)
ocr_df = ocr_df.dropna().reset_index(drop=True)
ocr_df

In [ ]:
draw = ImageDraw.Draw(image)

for idx, row in ocr_df.iterrows():
    x, y, w, h = row['left'], row['top'], row['width'], row['height']  # the row comes in (left, top, width, height) format
    draw.rectangle([x, y, x + w, y + h], width=2)

image

In [ ]:
%timeit
import easyocr

# specify languages and other configs
reader = easyocr.Reader(['en', 'uk'])

result = reader.readtext('/home/vdubyna/PycharmProjects/pdf-data-extraction/photo_2023-02-23_20-36-25.jpg', allowlist='АБВГДЕЄЇІЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЮЯ-#')
result

In [ ]:
from PIL import Image, ImageDraw, ImageFont
image = Image.open('/home/vdubyna/PycharmProjects/pdf-data-extraction/photo_2023-02-23_20-36-25.jpg')

draw = ImageDraw.Draw(image)
for res in result:
    top_left = tuple(res[0][0]) # top left coordinates as tuple
    bottom_right = tuple(res[0][2]) # bottom right coordinates as tuple
    # draw rectangle on image
    draw.rectangle([top_left[0], top_left[1], bottom_right[0], bottom_right[1]], width=2)

image